## 1. Setup and Dependencies

### 1.1 Import Libraries

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

## 2. MLflow Configuration

In [2]:
# Set the remote tracking server URI
mlflow.set_tracking_uri("http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/")

# Set or create an experiment for this specific tuning phase
mlflow.set_experiment("TfIdf Bigram max_features - Exp 3")

2025/11/15 19:37:56 INFO mlflow.tracking.fluent: Experiment with name 'TfIdf Bigram max_features - Exp 3' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlfow-bucket-2025/15', creation_time=1763215676144, experiment_id='15', last_update_time=1763215676144, lifecycle_stage='active', name='TfIdf Bigram max_features - Exp 3', tags={}>

## 3. Data Loading and Preparation

In [3]:
# Load the preprocessed data and ensure no NaN values remain in the text column
df = pd.read_csv('../data/reddit_preprocessing.csv').dropna(subset=['clean_comment'])

print(f"Data shape after cleaning: {df.shape}")

Data shape after cleaning: (36662, 2)


## 4. Experiment Function (TF-IDF Bigram)

In [4]:
def run_experiment_tfidf_max_features(max_features):
    """Runs a TF-IDF Bigram experiment with a specific max_features value and logs to MLflow."""
    
    ngram_range = (1, 2)  # Bigram setting (Unigrams and Bigrams)

    # Step 1: Initialize Vectorizer
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    # Step 2: Data Split (Stratified split to maintain class balance)
    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], 
                                                              test_size=0.2, random_state=42, stratify=df['category'])

    # Step 3: Vectorization
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    # Step 4: MLflow Run
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"TFIDF_Bigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_tuning")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Bigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters (Fixed for this experiment)
        n_estimators = 200
        max_depth = 15
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train_vec, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test_vec)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report details
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # 6. Log confusion matrix plot
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Bigrams, max_features={max_features}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # 7. Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_bigrams_{max_features}")
        
        print(f"Completed run: max_features={max_features}. Accuracy: {accuracy:.4f}")

## 5. Execute Experiments with Varying `max_features`

In [5]:
# Test various max_features values to find the optimal feature dimensionality
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)

2025/11/15 19:38:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/15 19:38:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Completed run: max_features=1000. Accuracy: 0.6626
🏃 View run TFIDF_Bigrams_max_features_1000 at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/15/runs/412828c3d63944a0a45c2340f06f4150
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/15


2025/11/15 19:40:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/15 19:40:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Completed run: max_features=2000. Accuracy: 0.6508
🏃 View run TFIDF_Bigrams_max_features_2000 at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/15/runs/9055790f3f0f4f80baf2e9fca0398922
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/15


2025/11/15 19:41:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/15 19:41:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Completed run: max_features=8000. Accuracy: 0.6475
🏃 View run TFIDF_Bigrams_max_features_8000 at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/15/runs/79d50537014147ab8badfd3414b460a2
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/15


2025/11/15 19:49:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/15 19:49:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Completed run: max_features=9000. Accuracy: 0.6430
🏃 View run TFIDF_Bigrams_max_features_9000 at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/15/runs/33c178536c114e4595496394a0f37bb5
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/15


2025/11/15 19:50:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/15 19:50:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Completed run: max_features=10000. Accuracy: 0.6394
🏃 View run TFIDF_Bigrams_max_features_10000 at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/15/runs/edb899e537104ea18c3fb29aae3d1886
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/15


## 6. Conclusion and Next Steps
Review the MLflow UI to compare the `accuracy` and `f1-score` across the different `max_features` runs for the TF-IDF Bigram model. Select the optimal `max_features` value to carry forward to the next stage of model tuning, which typically involves addressing class imbalance (Experiment 4).

In [7]:
import ast 

# Define the experiment type you want to compare
EXPERIMENT_TYPE = 'feature_tuning'

# --- 1. Query MLflow for all relevant runs ---
# Querying for runs specifically tagged with 'feature_tuning'
runs = mlflow.search_runs(
    filter_string=f"tags.experiment_type = '{EXPERIMENT_TYPE}'"
)

# --- 2. Extract and structure the metrics ---

comparison_data = []

# Iterate through the retrieved runs
for _, run in runs.iterrows():
    run_id = run['run_id']
    
    # Extract parameters, focusing only on the one being tuned
    max_features = run['params.vectorizer_max_features']
    
    # Extract metrics
    row = {
        'Max Features': int(max_features), # Convert to integer for proper sorting later
        'Accuracy (Overall)': run['metrics.accuracy'],
        # Metrics for the Minority Class (-1) - The critical metrics
        '-1_Precision': run.get('metrics.-1_precision', 0.0),
        '-1_Recall': run.get('metrics.-1_recall', 0.0),
        '-1_F1-Score': run.get('metrics.-1_f1-score', 0.0),
        'Run ID': run_id
    }
    comparison_data.append(row)

# --- 3. Create the comparison DataFrame ---
df_comparison = pd.DataFrame(comparison_data)

# Reorder columns (removing N-gram Range)
df_comparison = df_comparison[['Max Features', 'Accuracy (Overall)', '-1_Precision', '-1_Recall', '-1_F1-Score', 'Run ID']]

# Sort the DataFrame first by the critical F1-Score, then by Max Features (ascending)
df_comparison = df_comparison.sort_values(by=['-1_F1-Score', 'Max Features'], ascending=[False, True])

# --- 4. Print the final comparison table ---
print("--- Detailed Comparison of TF-IDF Max Features Tuning (Sorted by -1_F1-Score) ---")
print(f"Base Configuration: TF-IDF, N-gram Range (1, 2)")
print("-" * 75)
print(df_comparison.to_markdown(index=False, floatfmt=".4f"))

# --- 5. Highlight the Best Method ---
if not df_comparison.empty:
    best_method_row = df_comparison.iloc[0]
    print("\n" + "="*80)
    print("🥇 Optimal Feature Tuning (Based on Minority Class F1-Score):")
    print(f"Vectorization Method: TF-IDF Bigrams")
    print(f"Optimal Max Features: {best_method_row['Max Features']}")
    print(f"F1-Score (Minority Class): {best_method_row['-1_F1-Score']:.4f}")
    print("Rationale: This feature count maximizes the model's ability to distinguish the critical minority class.")
    print("="*80)
else:
    print(f"\nNo MLflow runs found for '{EXPERIMENT_TYPE}' experiment type.")

--- Detailed Comparison of TF-IDF Max Features Tuning (Sorted by -1_F1-Score) ---
Base Configuration: TF-IDF, N-gram Range (1, 2)
---------------------------------------------------------------------------
|   Max Features |   Accuracy (Overall) |   -1_Precision |   -1_Recall |   -1_F1-Score | Run ID                           |
|---------------:|---------------------:|---------------:|------------:|--------------:|:---------------------------------|
|           1000 |               0.6626 |         0.8791 |      0.1455 |        0.2496 | 412828c3d63944a0a45c2340f06f4150 |
|           3000 |               0.6544 |         0.9060 |      0.0642 |        0.1200 | 03d75788d418460e91a35f1b9805ae39 |
|           2000 |               0.6508 |         0.9565 |      0.0533 |        0.1010 | 9055790f3f0f4f80baf2e9fca0398922 |
|           4000 |               0.6536 |         0.9000 |      0.0491 |        0.0931 | 4e0e323f6df8470f84f6bf0400df42b5 |
|           5000 |               0.6529 |         